In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:42853")
client

Client Scheduler: tcp://127.0.0.1:42853 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 3 Cores: 3 Memory: 6.18 GB


In [35]:
import pandas as pd
import joblib

#Dask
import dask.dataframe as dd
from dask_ml.cluster import SpectralClustering
from dask_ml.cluster import KMeans
from dask_ml.model_selection import train_test_split, GridSearchCV, IncrementalSearchCV

#Sklearn
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn import svm, linear_model, tree
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor

#Local Files
import src.features_engineering as fte
import src.clustering as cl
import src.supervised_learning as sl

In [3]:
ratings = dd.read_csv('./input/ratings_small.csv')
genres_dummies = pd.read_csv('./input/genres_dummies.csv')

### Features Engineering Pipeline

In [4]:
%%time
ratings = (ratings.pipe(fte.addUserFeatures)
           .pipe(fte.addMoviesFeatures)
           .pipe(fte.filterbyRatingsAmount, min_rt=100, max_rt=500)
           .pipe(fte.addWeekdayColumns)
           .pipe(fte.addGenresDummies, genres_dummies=genres_dummies)
           .pipe(fte.popularityNormalizer)
           .pipe(fte.ratingsNormalizer)
          )

/home/castares/.local/lib/python3.6/site-packages/dask/dataframe/multi.py:1055: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indexes of each dataframes are 
 aligned. This assumption is not generally safe.
  "Concatenating dataframes with unknown divisions.\n"


CPU times: user 393 ms, sys: 21.7 ms, total: 415 ms
Wall time: 2.02 s


In [5]:
ratings.head()

,userId,movieId,GT,timestamp,user_rt_count,user_rt_mean,movie_rt_mean,popularity,weekday,weekday_6,...,Romance,Science Fiction,Sentai Filmworks,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western
0,4,1371,4.0,949810302,204,0.927937,0.567376,0.142415,6,1,...,0,0,0,0,0,0,0,0,0,0
1,19,1371,4.0,855193404,423,0.608286,0.567376,0.142415,3,0,...,0,0,0,0,0,0,0,0,0,0
2,21,1371,3.0,853852263,162,0.597246,0.567376,0.142415,1,0,...,0,0,0,0,0,0,0,0,0,0
3,22,1371,2.0,1131662302,220,0.506440,0.567376,0.142415,3,0,...,0,0,0,0,0,0,0,0,0,0
4,41,1371,3.5,1093886662,199,0.738916,0.567376,0.142415,0,0,...,0,0,0,0,0,0,0,0,0,0


# __Clustering__

In [37]:
users_genres = cl.dataScaling(cl.userGenresMatrix(ratings, genres_dummies))
users_genres_da = users_genres.to_dask_array(lengths=True)
users_genres.head()

,Action,Adventure,Animation,Aniplex,BROSTA TV,Carousel Productions,Comedy,Crime,Documentary,Drama,...,Romance,Science Fiction,Sentai Filmworks,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western
userId,,,,,,,,,,,,,,,,,,,,,
4,0.428571,0.261905,0.375,0.0,0.0,0.0,0.403226,0.326531,0.090909,0.421053,...,0.500000,0.545455,0.0,0.00,0.0,0.0,0.460317,0.0,0.333333,0.000000
8,0.102041,0.047619,0.125,0.0,0.0,0.0,0.145161,0.040816,0.272727,0.097744,...,0.145833,0.090909,0.0,0.00,0.0,0.0,0.047619,0.0,0.000000,0.142857
17,0.469388,0.500000,0.500,0.0,0.0,0.0,0.467742,0.530612,0.090909,0.518797,...,0.375000,0.575758,0.0,0.75,0.0,0.0,0.587302,0.0,0.444444,0.857143
19,0.959184,0.857143,0.875,0.0,0.0,0.0,0.935484,1.000000,0.545455,1.000000,...,0.937500,0.848485,0.0,0.00,0.0,0.0,1.000000,0.0,0.666667,0.428571
21,0.224490,0.190476,0.375,0.0,0.0,0.0,0.209677,0.326531,0.181818,0.270677,...,0.333333,0.242424,0.0,0.00,0.0,0.0,0.253968,0.0,0.111111,0.142857


In [39]:
%%time
#Spectral Clustering
clusters_number = 8
spcl = SpectralClustering(n_clusters=clusters_number, affinity='polynomial', n_jobs=-1)
clusters = spcl.fit_predict(users_genres_da)
clusters_index = cl.getClustersIndex(clusters, users_genres)
clusters_index.to_csv('./output/clusters-index/clusters-index-spcl-poly-8-*.csv')
clusters_index.compute()['cluster'].value_counts()

CPU times: user 12.4 s, sys: 1.05 s, total: 13.5 s
Wall time: 58 s


0    56
7    39
6    38
4    21
3    19
1    19
2    17
5    12
Name: clusters, dtype: int64

In [40]:
ratings = ratings.merge(clusters_index, left_on='userId', right_on='userId')

In [41]:
ratings.head()

,userId,movieId,GT,timestamp,user_rt_count,user_rt_mean,movie_rt_mean,popularity,weekday,weekday_6,...,Science Fiction,Sentai Filmworks,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western,clusters
0,4,1371,4.0,949810302,204,0.927937,0.567376,0.142415,6,1,...,0,0,0,0,0,0,0,0,0,6
1,4,2105,4.0,949896114,204,0.927937,0.661939,0.142415,0,0,...,0,0,0,0,0,0,0,0,0,6
2,4,2193,3.0,949896070,204,0.927937,0.600529,0.126935,0,0,...,0,0,0,0,0,0,0,0,0,6
3,4,153,4.0,949811346,204,0.927937,0.507321,0.396285,6,1,...,0,0,0,0,0,0,0,0,0,6
4,4,185,3.0,949920047,204,0.927937,0.578431,0.312693,0,0,...,1,0,0,0,0,0,0,0,0,6


In [ ]:
for e in range(clusters_number):
    

In [43]:
ratings.loc[ratings['clusters'] == 0].compute()

,userId,movieId,GT,timestamp,user_rt_count,user_rt_mean,movie_rt_mean,popularity,weekday,weekday_6,...,Science Fiction,Sentai Filmworks,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western,clusters
114,8,110,4.0,1154473268,116,0.738738,0.765595,0.702786,1,0,...,0,0,0,0,0,0,0,0,0,0
115,8,296,4.0,1154465380,116,0.738738,0.834705,1.000000,1,0,...,1,0,0,0,0,1,0,0,0,0
116,8,457,4.5,1154400475,116,0.738738,0.767345,0.656347,1,0,...,0,0,0,0,0,0,0,0,0,0
117,8,527,5.0,1154400170,116,0.738738,0.845173,0.752322,1,0,...,0,0,0,0,0,0,0,0,0,0
118,8,593,4.5,1154464833,116,0.738738,0.808480,0.938080,1,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,671,5991,4.5,1064245387,115,0.758776,0.699346,0.154799,0,0,...,0,0,0,0,0,0,0,0,0,0
43,671,1090,4.0,1064891507,115,0.758776,0.763072,0.207430,1,0,...,1,0,0,0,0,1,0,0,0,0
44,671,1641,4.0,1063503954,115,0.758776,0.727431,0.195046,6,1,...,0,0,0,0,0,0,0,0,0,0
45,671,2359,4.0,1063503933,115,0.758776,0.807407,0.089783,6,1,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#KMeans

# __Supervised Learning__

## __Full Dataset__

### X & y definition

In [6]:
X_columns = ['user_rt_mean', 'movie_rt_mean', 'popularity', 'weekday_6', 'weekday_2',
       'weekday_3', 'weekday_0', 'weekday_1', 'weekday_5', 'weekday_4',
       'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music',
       'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War',
       'Western']
y_columns = ['GT']

In [7]:
X = ratings[X_columns].to_dask_array(lengths=True)
y = ratings[y_columns].to_dask_array(lengths=True)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,y.ravel(),test_size=0.2)

dask.array<reshape, shape=(17030,), dtype=float64, chunksize=(17030,), chunktype=numpy.ndarray>

### Model Selection

In [25]:
models = {
        "RandomForest": RandomForestRegressor(n_estimators=100, n_jobs=-1),
        "SGDRegressor": linear_model.SGDRegressor(max_iter=1000, tol=1e-3),
        "DecisionTree" : tree.DecisionTreeRegressor(random_state=0),
        "GradientBoostingRegressor": GradientBoostingRegressor(n_estimators=100),
        "AdaBoostRegressor" : AdaBoostRegressor(n_estimators=100)    
    }
with joblib.parallel_backend('dask'):
    %time _ = sl.mlmodelSelection(models, X_train, X_test, y_train, y_test)    

Training model: RandomForest
RSME 0.8648452207825646
MAE 0.6553761546892124
r2_score 0.2877580337425849
Training model: SGDRegressor
RSME 0.8407794681065314
MAE 0.6380550853436995
r2_score 0.3268451564892342
Training model: DecisionTree
RSME 1.1555815495705373
MAE 0.8367778299671207
r2_score -0.2716042710504285
Training model: GradientBoostingRegressor
RSME 0.830836763341942
MAE 0.6275367247904382
r2_score 0.3426719123466514
Training model: AdaBoostRegressor
RSME 0.8833270836355243
MAE 0.697723172515674
r2_score 0.25699134496044085
CPU times: user 5.31 s, sys: 544 ms, total: 5.86 s
Wall time: 27.3 s


In [23]:
#GradientBoostingRegressor
#Hyperparameters search
model = GradientBoostingRegressor()
params = {
    'loss' : ['ls', 'lad', 'huber', 'quantile']
    'n_estimators' : [100, 400, 700],
    'max_depth' : [3, 10, 15]
}
'n_estimators' : [100, 400, 700],
'max_depth' : [3, 10, 15]

with joblib.parallel_backend('dask'):
    search = GridSearchCV(model, params)
    search.fit(X_train, y_train)

In [24]:
pd.DataFrame(search.cv_results_)

,params,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,param_loss
0,{'loss': 'ls'},3.129208,0.150514,0.018100,0.002876,0.277191,0.279200,0.391601,0.315993,0.053464,1,ls
1,{'loss': 'lad'},3.552742,0.086878,0.018127,0.000311,0.230807,0.257951,0.380722,0.289821,0.065219,3,lad
2,{'loss': 'huber'},3.740558,0.133385,0.013904,0.003890,0.272147,0.275398,0.383456,0.310329,0.051721,2,huber
3,{'loss': 'quantile'},0.303404,0.044609,0.004473,0.000927,-1.691302,-1.693457,-1.941155,-1.775295,0.117274,4,quantile


In [27]:
#Final Training
with joblib.parallel_backend('dask'):
    gbr = GradientBoostingRegressor()
    gbr.fit(X_train, y_train)
    y_pred = gbr.predict(X_test)
    print("RSME", (mean_squared_error(y_test, y_pred)**0.5))
    print("MAE", mean_absolute_error(y_test, y_pred))
    print("r2_score", r2_score(y_test, y_pred))

RSME 0.8309072076279523
MAE 0.6275466594705585
r2_score 0.34256044166338984


In [34]:
#saving the model to a pickle
with open('./output/models/gbrdefaultpickle_file.joblib', 'wb') as gbr_file:  
    joblib.dump(gbr, gbr_file)

'/home/castares/ironhack/movie-recommender'